# Mapping the DropSeq data using STARsolo

In [3]:
star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    star_reference_dir='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna';
    sjdbgtf='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
        --runThreadN 36 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist None \
        --soloType CB_UMI_Simple \
        --soloCBstart  1 \
        --soloCBlen 12 \
        --soloUMIstart 13 \
        --soloUMIlen 8\
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [2]:
dir=demultiplexed_dropseq
mkdir $dir
for sample in fastq_dropseq/*_2.fastq.gz
do
    NAME=${sample%_2.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq \
        ${NAME}_2.fastq.gz \
        ${NAME}_1.fastq.gz \
        $dir/$OUT.bam
done

mkdir: cannot create directory ‘demultiplexed’: File exists
fastq_dropseq/SRR1853178
Jan 27 23:27:06 ..... started STAR run
Jan 27 23:27:06 ..... loading genome
Jan 27 23:27:22 ..... processing annotations GTF
Jan 27 23:27:23 ..... started mapping
Jan 28 00:08:03 ..... finished mapping
Jan 28 00:08:08 ..... started Solo counting
Jan 28 00:19:21 ..... finished Solo counting
Jan 28 00:19:21 ..... started sorting BAM
Jan 28 00:27:47 ..... finished successfully
fastq_dropseq/SRR1853179
Jan 28 00:28:00 ..... started STAR run
Jan 28 00:28:00 ..... loading genome
Jan 28 00:28:34 ..... processing annotations GTF
Jan 28 00:28:40 ..... started mapping
Jan 28 01:10:24 ..... finished mapping
Jan 28 01:10:30 ..... started Solo counting
Jan 28 01:22:25 ..... finished Solo counting
Jan 28 01:22:25 ..... started sorting BAM
Jan 28 01:30:54 ..... finished successfully
fastq_dropseq/SRR1853180
Jan 28 01:31:10 ..... started STAR run
Jan 28 01:31:10 ..... loading genome
Jan 28 01:31:42 ..... processing an

: 135

In [4]:
dir=demultiplexed_dropseq
mkdir $dir
for sample in fastq_dropseq/SRR1853184_2.fastq.gz
do
    NAME=${sample%_2.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq \
        ${NAME}_2.fastq.gz \
        ${NAME}_1.fastq.gz \
        $dir/$OUT.bam
done

mkdir: cannot create directory ‘demultiplexed_dropseq’: File exists
fastq_dropseq/SRR1853184
Jan 28 11:32:30 ..... started STAR run
Jan 28 11:32:31 ..... loading genome
Jan 28 11:33:00 ..... processing annotations GTF
Jan 28 11:33:07 ..... started mapping
Jan 28 12:13:02 ..... finished mapping
Jan 28 12:13:05 ..... started Solo counting
Jan 28 12:23:56 ..... finished Solo counting
Jan 28 12:23:56 ..... started sorting BAM
Jan 28 12:31:58 ..... finished successfully


Now, we can get a summary of the statistics below:

In [5]:
for sample in demultiplexed_dropseq/*/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

SRR1853178.Solo.out/Gene/Summary.csv
Number of Reads,510210716
Reads With Valid Barcodes,1
Sequencing Saturation,0.757257
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.62094
Reads Mapped to Genome: Unique+Multiple,0.545704
Reads Mapped to Genome: Unique,0.545704
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.372239
Reads Mapped to Transcriptome: Unique Genes,0.362322
Estimated Number of Cells,2629
Reads in Cells Mapped to Unique Genes,54743780
Fraction of Reads in Cells,0.296135
Mean Reads per Cell,20823
Median Reads per Cell,15973
UMIs in Cells,7773057
Mean UMI per Cell,2956
Median UMI per Cell,2202
Mean Genes per Cell,1651
Median Genes per Cell,1391
Total Genes Detected,22048

SRR1853179.Solo.out/Gene/Summary.csv
Number of Reads,524537760
Reads With Valid Barcodes,1
Sequencing Saturation,0.768773
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.620952
Reads Mapped to Genome: Unique+Multiple,0.543425
Reads Mapped to Genome: Unique,0.543425
Reads Mapped to Transcriptome: Uniq